# Наследование в Python

Наследование в Python
* Наследование классов
* Множественное наследование
* Вызов super()
* name mangling
* Композиция vs наследование

Зачем нужно наследование классов?
* Изменение поведения класса
* Расширение функционала класса

In [1]:
# Класс домашнего питомца
class Pet():
    def __init__(self, name=None):
        self.name = name


# Класс собак
class Dog(Pet): # В скобках указан родительский класс, его еще называют базовый или супер класс
    
    def __init__(self, name, breed=None):
        super().__init__(name) # Вызов инициализации родительского класса
        self.breed = breed

    def say(self):
        return '{}: waw!'.format(self.name)


dog = Dog('Шарик', 'Доберман')

print(dog.name)  # Шарик
print(dog.breed) # Доберман
print(dog.say()) # Шарик: waw!

Шарик
Доберман
Шарик: waw!


## Множественное наследование

In [2]:
import json

class ExportJSON():

    def to_json(self):
        return json.dumps({
            'name': self.name,
            'breed': self.breed
        })

class ExDog(Dog, ExportJSON): # Множественное наследование
    pass

# Создаем экземпляр класса ExDog
dog = ExDog('Белка', breed='Дворняжка')

# Просмотр всех значений атрибутов экземпляра класса
print(dog.__dict__)
print(dog.to_json())

{'name': 'Белка', 'breed': 'Дворняжка'}
{"name": "\u0411\u0435\u043b\u043a\u0430", "breed": "\u0414\u0432\u043e\u0440\u043d\u044f\u0436\u043a\u0430"}


## Наследование от object (issubclass)

Любой класс является наследником класса `object`:

In [3]:
issubclass(int, object)

True

In [4]:
issubclass(Dog, object)

True

In [5]:
issubclass(Dog, Pet)

True

In [6]:
issubclass(Dog, int)

False

## Объект является экземпляром класса? (isinstance)

In [7]:
isinstance(dog, Dog)

True

In [8]:
isinstance(dog, Pet)

True

In [9]:
isinstance(dog, object)

True

## Поиск атрибутов и методов объекта. Линеаризация класса

MRO - **M**ethod **R**esolution **O**rder.  
Показывает иерархию классов и последовательность поиска атрибутов и методов объекта.  
Например, в начале поиск атрибутов или методов будет в классе `ExDog`, затем в `Dog`, `Pet`, `ExportJSON` и наконец в `object`:

In [10]:
ExDog.__mro__

(__main__.ExDog, __main__.Dog, __main__.Pet, __main__.ExportJSON, object)

## Использование super()

In [11]:
class ExDog(Dog, ExportJSON):

    def __init__(self, name, breed=None):
        # Вызов метода super() без параметров. Метод __init__ ищется по MRO
        super().__init__(name)

class WoolenDog(Dog, ExportJSON):

    def __init__(self, name, breed=None):
        # Вызов метода super() с параметрами. Явное указание метода __init__ конкретного класса
        super(Dog, self).__init__(name)
        self.breed = 'Шерстяная собака породы {}'.format(breed)


dog = WoolenDog('Жучка', 'Такса')

print(dog.breed)

Шерстяная собака породы Такса


## Разрешение конфликта имен, name mangling

In [13]:
class Dog(Pet):

    def __init__(self, name, breed=None):
        super().__init__(name)
        self.__breed = breed  # 2 символа подчеркивания обозначает как приватный атрибут

    def say(self):
        return '{}: wow!'.format(self.name)

    def get_breed(self):
        return self.__breed


class ExDog(Dog, ExportJSON):

    def get_breed(self):
        # self.__breed - Этот атрибут недоступен, ошибка AttributeError
        return 'Порода: {} - {}'.format(self.name, self.__breed)


dog = ExDog('Фокс', 'Мопс')

print( dog.get_breed() ) # AttributeError: 'ExDog' object has no attribute '_ExDog__breed'

AttributeError: 'ExDog' object has no attribute '_ExDog__breed'

In [14]:
dog.__dict__

{'_Dog__breed': 'Мопс', 'name': 'Фокс'}

Что же произошло? Метод `dog.get_breed()` попытался обратится к атрибуту `__breed (_ExDog__breed)` объекта класса `ExDog`, но такого атрибута не нашел. В словаре экземпляра класса `ExDog` мы видим атрибут `__breed (_Dog__breed)`, который пренадлежащит объекту класса `Dog`.  
Python позволяет обращаться к приватным атрибутам класса, поэтому мы можем исправить код нашего класса, но
лучше этим не увлекаться:

In [16]:
class ExDog(Dog, ExportJSON):

    def get_breed(self):
        # Исправление с self.__breed на self._Dog__breed
        return 'Порода: {} - {}'.format(self.name, self._Dog__breed)


dog = ExDog('Фокс', 'Мопс')

print( dog.get_breed() )

Порода: Фокс - Мопс


## Итоги

* Обсудили наследование классов
* Обсудили множественное наследование
* Рассмотрели вызов super()
* Узнали про `name mangling` (приватные атрибуты)